# Project 2: Detroit Blight Classification - Exploratory Data Analysis

This notebook analyzes three Detroit datasets to identify potential blight/inhabitability labels and features:
1. **Blight Survey Data** - DLBA property condition assessments
2. **COD Layers CSV** - Addresses, Buildings, Parcels with tax/assessment data
3. **Geodatabase** - Spatial layers (requires GIS tools)

**Primary Goal**: Hunt for blight/inhabitability labels for machine learning classification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotting setup
plt.style.use('default')
sns.set_palette("husl")

print("📊 Detroit Blight Classification EDA - Ready to analyze!")

📊 Detroit Blight Classification EDA - Ready to analyze!


## 1. Blight Survey Data Analysis - HUNTING FOR LABELS 🎯

This is our most promising dataset for blight classification labels!

In [2]:
def analyze_blight_survey_data():
    """
    Analyze the Detroit Land Bank Authority blight survey data
    Focus on finding classification labels for blight/inhabitability
    """
    print("=" * 60)
    print("🏠 BLIGHT SURVEY DATA ANALYSIS - LABEL HUNTING")
    print("=" * 60)
    
    # Load the Excel file - Updated path for new location
    file_path = "../../data/blight_survey_data/20250527_DLBA_survey_data_UM_Detroit.xlsx"
    
    try:
        df = pd.read_excel(file_path)
        
        print(f"📋 Dataset Shape: {df.shape}")
        print(f"📊 Number of rows: {df.shape[0]:,}")
        print(f"📋 Number of columns: {df.shape[1]}")
        print()
        
        # Focus on potential LABEL columns
        label_candidates = [
            'FIELD_DETERMINATION', 'FINAL_DETERMINATION', 'FINAL_DETERMINATION_2',
            'OTHER_RESOLUTION_PATHWAYS_DETERMINATION', 'SURVEY_STATUS',
            'HAS_STRUCTURE', 'IS_OCCUPIED'
        ]
        
        print("🎯 POTENTIAL BLIGHT CLASSIFICATION LABELS:")
        print("=" * 50)
        
        for col in label_candidates:
            if col in df.columns:
                null_count = df[col].isnull().sum()
                null_pct = (null_count / len(df)) * 100
                unique_vals = df[col].nunique()
                
                print(f"\n🏷️  {col}:")
                print(f"   Unique values: {unique_vals}")
                print(f"   Missing: {null_count:,} ({null_pct:.1f}%)")
                
                if unique_vals <= 20 and unique_vals > 0:
                    print(f"   Values: {sorted(df[col].dropna().unique())}")
                    print(f"   Value counts:")
                    value_counts = df[col].value_counts(dropna=False)
                    for val, count in value_counts.head(10).items():
                        pct = (count / len(df)) * 100
                        print(f"     {val}: {count:,} ({pct:.1f}%)")
        
        # Look at condition assessment columns
        condition_cols = [
            'FACADE_SIDING_CONDITION', 'FIRE_DAMAGE_CONDITION', 'ROOF_CONDITION',
            'OPENINGS_CONDITION', 'IS_OPEN_TO_TRESPASS', 'PORCH_STEPS_CONDITION'
        ]
        
        print("\n\n🏚️ PROPERTY CONDITION FEATURES (for ML features):")
        print("=" * 50)
        
        for col in condition_cols:
            if col in df.columns:
                null_count = df[col].isnull().sum()
                null_pct = (null_count / len(df)) * 100
                unique_vals = df[col].nunique()
                
                print(f"\n🔧 {col}:")
                print(f"   Unique values: {unique_vals}")
                print(f"   Missing: {null_count:,} ({null_pct:.1f}%)")
                
                if unique_vals <= 15 and unique_vals > 0:
                    value_counts = df[col].value_counts(dropna=False)
                    for val, count in value_counts.head(5).items():
                        pct = (count / len(df)) * 100
                        print(f"     {val}: {count:,} ({pct:.1f}%)")
        
        print("\n📝 Column Information (All columns):")
        print("-" * 40)
        for i, col in enumerate(df.columns, 1):
            dtype = str(df[col].dtype)
            null_count = df[col].isnull().sum()
            null_pct = (null_count / len(df)) * 100
            print(f"{i:2d}. {col:<35} | {dtype:<15} | {null_count:>5} nulls ({null_pct:5.1f}%)")
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading blight survey data: {e}")
        return None

# Run the analysis
blight_df = analyze_blight_survey_data()

🏠 BLIGHT SURVEY DATA ANALYSIS - LABEL HUNTING
📋 Dataset Shape: (62824, 30)
📊 Number of rows: 62,824
📋 Number of columns: 30

🎯 POTENTIAL BLIGHT CLASSIFICATION LABELS:

🏷️  FIELD_DETERMINATION:
   Unique values: 9
   Missing: 14,316 (22.8%)
   Values: [' NAP (Salvage)', 'Extreme Evidence of Blight', 'No Action (Salvage)', 'Noticeable Evidence of Blight', 'ODM (Demo)', 'Other Resolution Pathways (Salvage)', 'Property in Pipeline', 'Significant Evidence of Blight', 'Vacant (Not Blighted)']
   Value counts:
     Noticeable Evidence of Blight: 23,443 (37.3%)
     nan: 14,316 (22.8%)
     Significant Evidence of Blight: 8,956 (14.3%)
     Vacant (Not Blighted): 7,714 (12.3%)
     No Action (Salvage): 3,700 (5.9%)
      NAP (Salvage): 2,940 (4.7%)
     Extreme Evidence of Blight: 1,594 (2.5%)
     ODM (Demo): 139 (0.2%)
     Property in Pipeline: 20 (0.0%)
     Other Resolution Pathways (Salvage): 2 (0.0%)

🏷️  FINAL_DETERMINATION:
   Unique values: 7
   Missing: 22,532 (35.9%)
   Values: [' 

## 2. COD Layers Analysis - More Label Hunting 🔍

Looking for blight indicators in the City of Detroit address, building, and parcel data

In [3]:
def analyze_cod_layers_csv():
    """
    Analyze the City of Detroit (COD) layers CSV data
    Hunt for blight/inhabitability indicators
    """
    print("\n" + "=" * 60)
    print("🏙️ COD LAYERS CSV DATA ANALYSIS - BLIGHT HUNTING")
    print("=" * 60)
    
    # Updated path for new location
    data_dir = "../../data/cod_layers_csv/20250728_CODLayers.csv"
    csv_files = ["Addresses.csv", "Buildings.csv", "Parcels2025.csv"]
    
    datasets = {}
    
    for csv_file in csv_files:
        file_path = os.path.join(data_dir, csv_file)
        dataset_name = csv_file.replace('.csv', '')
        
        print(f"\n🏢 {dataset_name.upper()} Dataset:")
        print("-" * 50)
        
        try:
            df = pd.read_csv(file_path, low_memory=False)
            datasets[dataset_name] = df
            
            print(f"📊 Dataset Shape: {df.shape}")
            print(f"📋 Rows: {df.shape[0]:,}, Columns: {df.shape[1]}")
            
            # Hunt for blight-related columns
            blight_keywords = ['status', 'condition', 'class', 'vacant', 'occupied', 'tax', 'value', 'assessment']
            
            potential_blight_cols = []
            for col in df.columns:
                for keyword in blight_keywords:
                    if keyword.lower() in col.lower():
                        potential_blight_cols.append(col)
                        break
            
            if potential_blight_cols:
                print(f"\n🎯 POTENTIAL BLIGHT INDICATORS ({len(potential_blight_cols)} found):")
                for col in potential_blight_cols:
                    unique_vals = df[col].nunique()
                    null_count = df[col].isnull().sum()
                    null_pct = (null_count / len(df)) * 100
                    
                    print(f"\n   🏷️ {col}:")
                    print(f"      Unique values: {unique_vals}, Missing: {null_pct:.1f}%")
                    
                    if unique_vals <= 20 and unique_vals > 0:
                        value_counts = df[col].value_counts(dropna=False)
                        for val, count in value_counts.head(5).items():
                            pct = (count / len(df)) * 100
                            print(f"        {val}: {count:,} ({pct:.1f}%)")
            
            # Show all columns for reference
            print(f"\n📝 All Columns ({df.shape[1]}):")    
            for i, col in enumerate(df.columns, 1):
                dtype = str(df[col].dtype)
                null_count = df[col].isnull().sum()
                null_pct = (null_count / len(df)) * 100
                blight_flag = "🎯" if col in potential_blight_cols else "  "
                print(f"{blight_flag}{i:2d}. {col:<30} | {dtype:<15} | {null_count:>6} nulls ({null_pct:5.1f}%)")
            
        except Exception as e:
            print(f"❌ Error loading {csv_file}: {e}")
    
    return datasets

# Run the analysis
cod_datasets = analyze_cod_layers_csv()


🏙️ COD LAYERS CSV DATA ANALYSIS - BLIGHT HUNTING

🏢 ADDRESSES Dataset:
--------------------------------------------------
📊 Dataset Shape: (490561, 12)
📋 Rows: 490,561, Columns: 12

📝 All Columns (12):
   1. OBJECTID                       | int64           |      0 nulls (  0.0%)
   2. addr_id                        | int64           |      0 nulls (  0.0%)
   3. unit_id                        | float64         | 366083 nulls ( 74.6%)
   4. bldg_id                        | float64         | 128409 nulls ( 26.2%)
   5. parcel_id                      | object          |    913 nulls (  0.2%)
   6. street_id                      | float64         |   1278 nulls (  0.3%)
   7. street_number                  | int64           |      0 nulls (  0.0%)
   8. unit_type                      | object          | 428115 nulls ( 87.3%)
   9. unit_number                    | object          | 412708 nulls ( 84.1%)
  10. streetname_id                  | float64         | 376732 nulls ( 76.8%)
  11. z

## 3. Deep Dive into Parcels Data - Tax and Assessment Indicators 💰

The Parcels2025 dataset likely has the richest blight indicators through tax status, property values, and conditions

In [4]:
# Deep dive into Parcels data for blight indicators
if 'Parcels2025' in cod_datasets:
    parcels_df = cod_datasets['Parcels2025']
    
    print("🏡 PARCELS 2025 - DEEP DIVE FOR BLIGHT LABELS")
    print("=" * 60)
    
    # Key columns that might indicate blight
    key_blight_cols = [
        'Tax Status', 'Tax Status Description', 'Property Class', 'Property Class Description',
        'Use Code', 'Use Code Description', 'Assessed Value', 'Taxable Value', 
        'Sale Price', 'Is Improved', 'Year Built', 'Building Style', 'Building Count'
    ]
    
    for col in key_blight_cols:
        if col in parcels_df.columns:
            print(f"\n🔍 {col}:")
            
            unique_vals = parcels_df[col].nunique()
            null_count = parcels_df[col].isnull().sum()
            null_pct = (null_count / len(parcels_df)) * 100
            
            print(f"   Unique: {unique_vals}, Missing: {null_pct:.1f}%")
            
            if unique_vals <= 50 and unique_vals > 0:
                print("   Top values:")
                value_counts = parcels_df[col].value_counts(dropna=False)
                for val, count in value_counts.head(10).items():
                    pct = (count / len(parcels_df)) * 100
                    print(f"     {val}: {count:,} ({pct:.1f}%)")
            elif parcels_df[col].dtype in ['int64', 'float64']:
                print(f"   Stats: Mean={parcels_df[col].mean():.2f}, Median={parcels_df[col].median():.2f}")
                print(f"   Range: {parcels_df[col].min():.2f} to {parcels_df[col].max():.2f}")
    
    # Look for zero/low value properties (potential blight indicator)
    print("\n💰 LOW VALUE PROPERTIES (Potential Blight Indicators):")
    print("-" * 50)
    
    # Zero assessed value
    zero_assessed = (parcels_df['Assessed Value'] == 0).sum()
    zero_assessed_pct = (zero_assessed / len(parcels_df)) * 100
    print(f"Properties with $0 Assessed Value: {zero_assessed:,} ({zero_assessed_pct:.1f}%)")
    
    # Very low assessed value (under $1000)
    low_assessed = (parcels_df['Assessed Value'] < 1000).sum()
    low_assessed_pct = (low_assessed / len(parcels_df)) * 100
    print(f"Properties with <$1,000 Assessed Value: {low_assessed:,} ({low_assessed_pct:.1f}%)")
    
    # Zero sale price
    zero_sale = (parcels_df['Sale Price'] == 0).sum()
    zero_sale_pct = (zero_sale / len(parcels_df)) * 100
    print(f"Properties with $0 Sale Price: {zero_sale:,} ({zero_sale_pct:.1f}%)")

else:
    print("❌ Parcels2025 data not available for deep dive")

🏡 PARCELS 2025 - DEEP DIVE FOR BLIGHT LABELS

🔍 Tax Status:
   Unique: 24, Missing: 0.0%
   Top values:
     TAXABLE: 298,133 (78.8%)
     EXEMPT (211.7GG): 61,925 (16.4%)
     EXEMPT (211.7M): 8,977 (2.4%)
     EXEMPT (211.7S): 3,468 (0.9%)
     EXEMPT (125.1415A): 1,595 (0.4%)
     EXEMPT (211.7L): 1,098 (0.3%)
     EXEMPT (211.7O): 931 (0.2%)
     EXEMPT (211.7N): 483 (0.1%)
     EXEMPT (211.7Z): 404 (0.1%)
     EXEMPT (OTHER): 385 (0.1%)

🔍 Tax Status Description:
   Unique: 24, Missing: 0.0%
   Top values:
     TAXABLE: 298,133 (78.8%)
     PROPERTY HELD BY LAND BANK FAST TRACK AUTHORITY: 61,925 (16.4%)
     COUNTY, TOWNSHIP, CITY, VILLAGE, SCHOOL DISTRICT, PARKS: 8,977 (2.4%)
     HOUSES OF PUBLIC WORSHIP; PARSONAGE: 3,468 (0.9%)
     EXEMPTION FROM HOUSING PROJECTS: 1,595 (0.4%)
     STATE PROPERTY: 1,098 (0.3%)
     NON-PROFIT CHARITABLE INSTITUTION: 931 (0.2%)
     NON-PROFIT THEATER, LIBRARY, EDUCATIONAL, SCIENTIFIC INSTITUTION: 483 (0.1%)
     SCHOOL: 404 (0.1%)
      : 385 

## 4. Label Creation Strategy 🎯

Based on our analysis, let's create potential blight classification labels

In [5]:
def create_blight_labels(blight_df, parcels_df):
    """
    Create potential blight classification labels from the datasets
    """
    print("🎯 CREATING BLIGHT CLASSIFICATION LABELS")
    print("=" * 50)
    
    labels_summary = {}
    
    if blight_df is not None:
        print("\n🏠 BLIGHT SURVEY LABELS:")
        print("-" * 30)
        
        # Option 1: Final Determination as primary label
        if 'FINAL_DETERMINATION' in blight_df.columns:
            valid_determinations = blight_df['FINAL_DETERMINATION'].dropna()
            labels_summary['final_determination'] = {
                'count': len(valid_determinations),
                'unique_values': valid_determinations.nunique(),
                'distribution': valid_determinations.value_counts().to_dict()
            }
            print(f"✅ FINAL_DETERMINATION: {len(valid_determinations):,} valid labels")
            print(f"   Categories: {valid_determinations.nunique()}")
            
        # Option 2: Field Determination as backup
        if 'FIELD_DETERMINATION' in blight_df.columns:
            valid_field = blight_df['FIELD_DETERMINATION'].dropna()
            labels_summary['field_determination'] = {
                'count': len(valid_field),
                'unique_values': valid_field.nunique(),
                'distribution': valid_field.value_counts().to_dict()
            }
            print(f"✅ FIELD_DETERMINATION: {len(valid_field):,} valid labels")
            print(f"   Categories: {valid_field.nunique()}")
    
    if 'Parcels2025' in cod_datasets:
        print("\n🏡 PARCELS-BASED LABELS:")
        print("-" * 30)
        
        # Create binary blight indicator based on low property values
        low_value_threshold = 1000
        blight_indicator = (parcels_df['Assessed Value'] < low_value_threshold) & (parcels_df['Assessed Value'] > 0)
        
        blight_count = blight_indicator.sum()
        blight_pct = (blight_count / len(parcels_df)) * 100
        
        labels_summary['low_value_blight'] = {
            'count': len(parcels_df),
            'blight_properties': blight_count,
            'blight_percentage': blight_pct
        }
        
        print(f"✅ LOW VALUE BLIGHT (< ${low_value_threshold}): {blight_count:,} properties ({blight_pct:.1f}%)")
        
        # Tax status based labels
        if 'Tax Status Description' in parcels_df.columns:
            tax_status_counts = parcels_df['Tax Status Description'].value_counts()
            labels_summary['tax_status'] = tax_status_counts.to_dict()
            print(f"✅ TAX STATUS LABELS: {len(tax_status_counts)} categories")
            for status, count in tax_status_counts.head(5).items():
                pct = (count / len(parcels_df)) * 100
                print(f"   {status}: {count:,} ({pct:.1f}%)")
    
    print("\n🎯 RECOMMENDED LABELING STRATEGY:")
    print("=" * 40)
    print("1. PRIMARY: Use FINAL_DETERMINATION from blight survey (highest quality)")
    print("2. SECONDARY: Use FIELD_DETERMINATION as backup/additional data")
    print("3. FEATURES: Combine with property conditions, tax status, assessed values")
    print("4. AUGMENT: Create synthetic labels using low assessed values")
    
    return labels_summary

# Create the labels
if blight_df is not None and 'Parcels2025' in cod_datasets:
    label_analysis = create_blight_labels(blight_df, cod_datasets['Parcels2025'])
else:
    print("❌ Cannot create labels - missing required datasets")

🎯 CREATING BLIGHT CLASSIFICATION LABELS

🏠 BLIGHT SURVEY LABELS:
------------------------------
✅ FINAL_DETERMINATION: 40,292 valid labels
   Categories: 7
✅ FIELD_DETERMINATION: 48,508 valid labels
   Categories: 9

🏡 PARCELS-BASED LABELS:
------------------------------
✅ LOW VALUE BLIGHT (< $1000): 29,650 properties (7.8%)
✅ TAX STATUS LABELS: 24 categories
   TAXABLE: 298,133 (78.8%)
   PROPERTY HELD BY LAND BANK FAST TRACK AUTHORITY: 61,925 (16.4%)
   COUNTY, TOWNSHIP, CITY, VILLAGE, SCHOOL DISTRICT, PARKS: 8,977 (2.4%)
   HOUSES OF PUBLIC WORSHIP; PARSONAGE: 3,468 (0.9%)
   EXEMPTION FROM HOUSING PROJECTS: 1,595 (0.4%)

🎯 RECOMMENDED LABELING STRATEGY:
1. PRIMARY: Use FINAL_DETERMINATION from blight survey (highest quality)
2. SECONDARY: Use FIELD_DETERMINATION as backup/additional data
3. FEATURES: Combine with property conditions, tax status, assessed values
4. AUGMENT: Create synthetic labels using low assessed values


## 5. Geodatabase Information 🗺️

In [6]:
def analyze_geodatabase_info():
    """
    Analyze the geodatabase structure (limited without specialized GIS tools)
    """
    print("\n" + "=" * 60)
    print("🗺️ GEODATABASE ANALYSIS")
    print("=" * 60)
    
    # Updated path for new location
    gdb_path = "../../data/cod_layers_gdb/CODBaseUnitLayers.gdb"
    
    if os.path.exists(gdb_path):
        print(f"📍 Geodatabase found at: {gdb_path}")
        
        # Get file information
        files = os.listdir(gdb_path)
        print(f"📁 Number of files in geodatabase: {len(files)}")
        
        # Categorize files by extension
        extensions = {}
        for file in files:
            ext = os.path.splitext(file)[1] if '.' in file else 'no_extension'
            extensions[ext] = extensions.get(ext, 0) + 1
        
        print("\n📋 File types in geodatabase:")
        for ext, count in sorted(extensions.items()):
            print(f"  {ext}: {count} files")
        
        # Get total size
        total_size = sum(os.path.getsize(os.path.join(gdb_path, f)) for f in files)
        print(f"\n💾 Total geodatabase size: {total_size / (1024*1024):.2f} MB")
        
        print("\n⚠️  Note: This is an ESRI geodatabase (.gdb) which requires")
        print("   specialized GIS software (like ArcGIS or QGIS with GDAL) for full analysis.")
        print("   Consider using geopandas or arcpy for detailed geodatabase exploration.")
        
    else:
        print("❌ Geodatabase not found!")
    
    return None

# Analyze geodatabase
gdb_info = analyze_geodatabase_info()


🗺️ GEODATABASE ANALYSIS
📍 Geodatabase found at: ../../data/cod_layers_gdb/CODBaseUnitLayers.gdb
📁 Number of files in geodatabase: 70

📋 File types in geodatabase:
  .atx: 24 files
  .freelist: 1 files
  .gdbindexes: 11 files
  .gdbtable: 12 files
  .gdbtablx: 12 files
  .horizon: 4 files
  .spx: 4 files
  no_extension: 2 files

💾 Total geodatabase size: 284.45 MB

⚠️  Note: This is an ESRI geodatabase (.gdb) which requires
   specialized GIS software (like ArcGIS or QGIS with GDAL) for full analysis.
   Consider using geopandas or arcpy for detailed geodatabase exploration.


## 6. Summary & Next Steps 📋

Key findings for Project 2 blight classification

In [7]:
print("\n" + "=" * 80)
print("📊 PROJECT 2 BLIGHT CLASSIFICATION - SUMMARY")
print("=" * 80)

if blight_df is not None:
    print(f"✅ Blight Survey Data: {blight_df.shape[0]:,} rows, {blight_df.shape[1]} columns")
    
    # Check key label columns
    final_det_count = blight_df['FINAL_DETERMINATION'].notna().sum() if 'FINAL_DETERMINATION' in blight_df.columns else 0
    field_det_count = blight_df['FIELD_DETERMINATION'].notna().sum() if 'FIELD_DETERMINATION' in blight_df.columns else 0
    
    print(f"   🎯 FINAL_DETERMINATION labels: {final_det_count:,}")
    print(f"   🎯 FIELD_DETERMINATION labels: {field_det_count:,}")

if cod_datasets:
    for name, df in cod_datasets.items():
        print(f"✅ COD {name}: {df.shape[0]:,} rows, {df.shape[1]} columns")

print("✅ Geodatabase: 284.45 MB spatial data")

print("\n🎯 BLIGHT CLASSIFICATION RECOMMENDATIONS:")
print("=" * 50)
print("1. **PRIMARY LABELS**: Use FINAL_DETERMINATION from blight survey")
print("2. **BACKUP LABELS**: Use FIELD_DETERMINATION for additional training data")
print("3. **FEATURES**: Property conditions, tax status, assessed values, building age")
print("4. **AUGMENTATION**: Create binary labels from low property values (<$1000)")
print("5. **SPATIAL**: Use geodatabase for neighborhood context features")

print("\n🚀 NEXT STEPS:")
print("- Clean and preprocess the blight survey labels")
print("- Join datasets on parcel_id for feature engineering")
print("- Create train/test splits preserving spatial distribution")
print("- Build baseline models (Random Forest, XGBoost)")
print("- Explore advanced models with spatial features")

print("\n📊 Dataset Ready for Machine Learning! 🤖")


📊 PROJECT 2 BLIGHT CLASSIFICATION - SUMMARY
✅ Blight Survey Data: 62,824 rows, 30 columns
   🎯 FINAL_DETERMINATION labels: 40,292
   🎯 FIELD_DETERMINATION labels: 48,508
✅ COD Addresses: 490,561 rows, 12 columns
✅ COD Buildings: 409,301 rows, 8 columns
✅ COD Parcels2025: 378,366 rows, 52 columns
✅ Geodatabase: 284.45 MB spatial data

🎯 BLIGHT CLASSIFICATION RECOMMENDATIONS:
1. **PRIMARY LABELS**: Use FINAL_DETERMINATION from blight survey
2. **BACKUP LABELS**: Use FIELD_DETERMINATION for additional training data
3. **FEATURES**: Property conditions, tax status, assessed values, building age
4. **AUGMENTATION**: Create binary labels from low property values (<$1000)
5. **SPATIAL**: Use geodatabase for neighborhood context features

🚀 NEXT STEPS:
- Clean and preprocess the blight survey labels
- Join datasets on parcel_id for feature engineering
- Create train/test splits preserving spatial distribution
- Build baseline models (Random Forest, XGBoost)
- Explore advanced models with spat

In [9]:
blight_df['FINAL_DETERMINATION'].value_counts()

FINAL_DETERMINATION
Property in Pipeline                   14547
Other Resolution Pathways (Salvage)    11234
Vacant (Not Blighted)                   4940
No Action (Salvage)                     4460
 NAP (Salvage)                          4451
ODM (Demo)                               634
Noticeable Evidence of Blight             26
Name: count, dtype: int64